In [5]:
from auton_survival import datasets

outcomes, features = datasets.load_support()

In [6]:
from auton_survival.preprocessing import Preprocessor

cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp',
             'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph',
             'glucose', 'bun', 'urine', 'adlp', 'adls']

features = Preprocessor().fit_transform(features, cat_feats=cat_feats, num_feats=num_feats)

import numpy as np

horizons = [0.25, 0.5, 0.75]
times = np.quantile(outcomes.time[outcomes.event == 1], horizons).tolist()

x, t, e = features.values.astype(float), outcomes.time.values.astype(float), outcomes.event.values.astype(float)

n = len(x)

tr_size = int(n * 0.70)
vl_size = int(n * 0.10)
te_size = int(n * 0.20)

print(tr_size, vl_size, te_size)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size + vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size + vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size + vl_size]

218 31 62


In [7]:
x_train[0].shape

(2, 25)

In [8]:
models = []

from auton_survival.models.dpsm import DeepDP
from auton_survival.models.dsm import DeepSurvivalMachines

# model = DeepDP(k=3,
#                distribution='LogNormal',
#                layers=[100])
model = DeepSurvivalMachines(
    k=4,
    distribution="LogNormal",
    layers=[100]
)
# The fit method is called to train the model
model.fit(x_train, t_train, e_train, iters=100, learning_rate=0.001)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
trained_weights = model.trained_weights
print(trained_weights.shape)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

figs, axes = plt.subplots(1, 3, figsize=(15, 5))
iter_idx = [14, 15, 16]

for idx in range(3):
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 0], fill=True, ax=axes[idx])
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 1], fill=True, ax=axes[idx])
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 2], fill=True, ax=axes[idx])
    sns.kdeplot(trained_weights[iter_idx[idx]][:, 3], fill=True, ax=axes[idx])
    axes[idx].set_title(f'Iter {iter_idx[idx]}')
    # axes[idx].set_xlim(0.124, 0.126)

# sns.kdeplot(trained_weights[iter_idx][:, 0], fill=True)
# sns.kdeplot(trained_weights[iter_idx][:, 1], fill=True)
# sns.kdeplot(trained_weights[iter_idx][:, 2], fill=True)
# plt.title(f'Iter {iter_idx}')